In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.graphs import Neo4jGraph
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="test1234")
graph.query(
    """
MATCH (n)
DETACH DELETE n
"""
)
graph.refresh_schema()
print(graph.schema)

Node properties are the following:

Relationship properties are the following:

The relationships are the following:



In [3]:
from loader.radiopaedia import RadioWebLoader
search_query = "Atelectasis"
radio_raw_documents = RadioWebLoader(search_query, only_first=True).load()
wiki_raw_documents = [WikipediaLoader(query="Atelectasis").load()[0]]
raw_documents = radio_raw_documents + wiki_raw_documents

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [4]:
len(radio_raw_documents), len(wiki_raw_documents)

(1, 1)

In [5]:
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents(
    raw_documents
)  # should I only take the first? which is the most relevant?

In [6]:
len(raw_documents)

2

In [7]:
from tqdm import tqdm
from aug.graph_doc import (
    get_extraction_chain,
    data_to_graph_doc,
    chain_run,
    add_graph_documents,
)
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

allowed_nodes = None
allowed_rels = None
# allowed_nodes = ["Symptom", "Disease"]
# allowed_rels = ["CAN_CAUSE", "DESCRIBE", "HAS"]

extract_chain = get_extraction_chain(llm, allowed_nodes, allowed_rels)
gds = []

for d in tqdm(documents, total=len(documents)):
    data = chain_run(extract_chain, d.page_content)
    # data = extract_chain.run(d.page_content)
    # graph_document = GraphDocument(
    #     nodes=[map_to_base_node(node) for node in data.nodes],
    #     relationships=[map_to_base_relationship(rel) for rel in data.rels],
    #     source=d,
    # )
    graph_document = data_to_graph_doc(data, d)
    # add_graph_document(graph, graph_document)
    gds.append(graph_document)

graph = add_graph_documents(graph, gds)
graph.refresh_schema()
print(graph.schema)

# 12m46.2s

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
 33%|███▎      | 1/3 [00:02<00:04,  2.19s/it]Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
).
100%|██████████| 3/3 [02:15<00:00, 45.23s/it]


Node properties are the following:
Condition {name: STRING, id: STRING, affects: STRING, description: STRING, definition: STRING, unilateral: STRING},Symptom {description: STRING, id: STRING, name: STRING},Cause {description: STRING, name: STRING, id: STRING},Risk factor {description: STRING, id: STRING, name: STRING},Type {name: STRING, id: STRING, description: STRING},Procedure {id: STRING, name: STRING},Medicalcondition {summary: STRING, id: STRING, name: STRING},Body part {id: STRING, name: STRING},Finding {name: STRING, id: STRING},Presentation {id: STRING, name: STRING},Event {name: STRING, id: STRING},Injury {id: STRING, name: STRING},Medical specialty {name: STRING, id: STRING}
Relationship properties are the following:

The relationships are the following:
(:Condition)-[:CAN_CAUSE]->(:Symptom),(:Condition)-[:CAN_CAUSE]->(:Condition),(:Condition)-[:IS_A_TYPE_OF]->(:Type),(:Condition)-[:IS_A]->(:Condition),(:Cause)-[:CAN_CAUSE]->(:Condition),(:Risk factor)-[:INCREASES_LIKELIHOOD

In [8]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    validate_cypher=True,  # Validate relationship directions
    verbose=True,
)

In [9]:
lesion = "Atelectasis"
questions = [
    f"What is {lesion}?",
    f"What are the symptoms associated with {lesion}?",
    f"What can cause {lesion}?",
    f"What are the patient’s symptoms that are relevant for {lesion}?",
    f"What are the relevant clinical signs for the etiological diagnosis of {lesion}?",
    f"What are the relevant laboratory data for the etiological diagnosis of {lesion}?",
    f"What are the relevant clinical characteristics for the etiological diagnosis of {lesion}",
    f"What are the patient’s personal relevant history for the etiological diagnosis of {lesion}",
]

In [10]:
res_dict = {}
for q in questions:
    try:
        res = chain.run(q)
        res_dict[q] = res.strip()
    except:
        res_dict[q] = "Generated Cypher Statement is not valid"
# res = chain.run("What is Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Condition {name: "Atelectasis"})
RETURN c
Full Context:
[{'c': {'name': 'Atelectasis', 'affects': 'part or all of one lung', 'description': 'Atelectasis is a condition where the alveoli are deflated down to little or no volume, as distinct from pulmonary consolidation, in which they are filled with liquid. It is often referred to informally as a collapsed lung, although more accurately it usually involves only a partial collapse, and that ambiguous term is also informally used for a fully collapsed lung caused by a pneumothorax. It is a very common finding in chest X-rays and other radiological studies, and may be caused by normal exhalation or by various medical conditions. Although frequently described as a collapse of lung tissue, atelectasis is not synonymous with a pneumothorax, which is a more specific condition that can cause atelectasis.', 'definition': 'The partial collapse or closure of a lung resulting i

In [11]:
for q in questions:
    print(res_dict[q])

Atelectasis is a condition where part or all of one lung is affected. It is characterized by the deflation of the alveoli, which are the tiny air sacs in the lungs. This condition is often referred to as a collapsed lung, although it usually involves only a partial collapse. Atelectasis is a common finding in chest X-rays and other radiological studies. It can be caused by normal exhalation or various medical conditions. It is important to note that atelectasis is not synonymous with a pneumothorax, which is a more specific condition that can cause atelectasis.
The symptoms associated with Atelectasis include Cyanosis and Increased Heart Rate.
Pleural effusion can cause atelectasis.
The patient's symptoms that are relevant for Atelectasis include Cyanosis and Increased Heart Rate.
The relevant clinical signs for the etiological diagnosis of Atelectasis include Cyanosis and Increased Heart Rate.
The relevant laboratory data for the etiological diagnosis of Atelectasis include Cyanosis a

In [12]:
raise StopIteration()

StopIteration: 

In [ ]:
chain.run("What is Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Condition {name: 'Atelectasis'})
RETURN c
Full Context:
[{'c': {'name': 'Atelectasis', 'causedBy': 'normal exhalation or various medical conditions', 'affects': 'part or all of one lung', 'description': 'A condition where the alveoli are deflated down to little or no volume, as distinct from pulmonary consolidation, in which they are filled with liquid', 'definition': 'The collapse or closure of a lung resulting in reduced or absent gas exchange', 'unilateral': 'true', 'id': 'Atelectasis', 'alsoKnownAs': 'collapsed lung'}}]

> Finished chain.


'Atelectasis is a condition where part or all of one lung collapses or closes, resulting in reduced or absent gas exchange. It can be caused by normal exhalation or various medical conditions. The alveoli, which are small air sacs in the lungs, become deflated down to little or no volume. Atelectasis is also known as a collapsed lung.'

In [ ]:
chain.run("What are the symptoms of Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Condition {name: 'Atelectasis'})-[:HAS_SYMPTOM]->(s:Symptom)
RETURN s.name
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have the information to answer your question. It would be best to consult a medical professional for accurate information on the symptoms of Atelectasis."

In [ ]:
chain.run("What can cause Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Condition {name: 'Atelectasis'})-[:DUE_TO]->(cause:Cause)
RETURN cause.name
Full Context:
[]

> Finished chain.


'There are several factors that can cause atelectasis. Some common causes include blockage of the airways due to mucus, a foreign object, or a tumor, as well as lung diseases such as chronic obstructive pulmonary disease (COPD) or pneumonia. Additionally, certain medical procedures, such as anesthesia or prolonged bed rest, can also contribute to the development of atelectasis. It is important to consult with a healthcare professional for a proper diagnosis and treatment plan.'

In [ ]:
chain.run("What clinical features radiologists need for diagnosing Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Condition)-[:HAS]->(s:Section)-[:MENTIONS]->(f:Finding)
WHERE a.name = 'Atelectasis'
RETURN f.name
Full Context:
[]

> Finished chain.


'To diagnose Atelectasis, radiologists typically look for certain clinical features. These may include signs such as lung collapse, airway obstruction, and reduced lung volume. Additionally, they may also consider the presence of symptoms such as shortness of breath, coughing, and chest pain. By evaluating these clinical features, radiologists can make an accurate diagnosis of Atelectasis.'

In [ ]:
chain.run("what are radiographic features of Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have the information to answer your question about the radiographic features of Atelectasis."